# BiLSTM을 이용한 영화 리뷰 감정 분석
Word2Vec을 통해 임베딩 된 네이버 영화 리뷰 데이터를 BiLSTM을 통해 긍정, 부정을 분류해주는 예제


In [2]:
import os
import tensorflow as tf
import Bi_LSTM
import Word2Vec
import gensim
import numpy as np

In [3]:
WORD2VEC_MODEL = "../data/Word2Vec.model"
BILSTM_MODEL = "../data/BiLSTM_model.ckpt"

In [4]:
W2V = Word2Vec.Word2Vec()

In [5]:
Batch_size = 1
Vector_size = 300
Maxseq_length = 95   ## Max length of training data
learning_rate = 0.001
lstm_units = 128
num_class = 2
keep_prob = 1.0

In [6]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

In [7]:
BiLSTM = Bi_LSTM.Bi_LSTM(lstm_units, num_class, keep_prob)

In [8]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

In [9]:
#tf.nn.softmax?

In [10]:
#tf.variable_scope?

In [11]:
prediction = tf.nn.softmax(logits)

In [12]:
def Convert2Vec(model_name, sentence):    
    word_vec = []
    sub = []
    model = gensim.models.word2vec.Word2Vec.load(model_name)
    for word in sentence:
        if(word in model.wv.vocab):
            sub.append(model.wv[word])
        else:
            sub.append(np.random.uniform(-0.25,0.25,300)) ## used for OOV words
    word_vec.append(sub)
    return word_vec

In [13]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = BILSTM_MODEL

In [14]:
sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)

INFO:tensorflow:Restoring parameters from ../data/BiLSTM_model.ckpt


In [25]:
def Grade(sentence):

    tokens = W2V.tokenize(sentence)
    
    embedding = Convert2Vec(WORD2VEC_MODEL, tokens)
    zero_pad = W2V.Zero_padding(embedding, Batch_size, Maxseq_length, Vector_size)
    
    global sess
    #result =  sess.run(tf.argmax(prediction,1), feed_dict = {X: zero_pad , seq_len: [len(tokens)] })
    result =  sess.run(prediction, feed_dict = {X: zero_pad , seq_len: [len(tokens)] })
    category = sess.run(tf.argmax(prediction,1), feed_dict = {X: zero_pad , seq_len: [len(tokens)] })
    
    if(category == 1):
        print("리뷰는 긍정입니다.", result)
    else:
        print("리뷰는 부정입니다.", result)
        
    print(np.shape(result))
    print(result[0][1])

In [26]:
Grade("재미있어요")

리뷰는 긍정입니다. [[0.12218965 0.8778103 ]]
(1, 2)
0.8778103


In [ ]:
while(1):
    s = input("문장을 입력하세요 : ")
    if(s == str(1)):
        break
    else:
        Grade(s)

문장을 입력하세요 : 재밌네요!?!?
리뷰는 긍정입니다. [[0.44553402 0.5544659 ]]
(1, 2)
0.5544659
